## Инициализация

In [57]:
import torch
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
import json
import pickle
import shutil
import os
from openai import OpenAI
import pandas as pd

In [58]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
HARD_PATH = r"..\4_merge_data\results\hard_skills_final.txt"
SOFT_PATH = r"..\4_merge_data\results\soft_skills_final.txt"
SKILLS_PATH = r"..\4_merge_data\results\merged_skills_final.csv"
WORKING_DIRECTORY = r"results"

model = SentenceTransformer(
    'ai-forever/FRIDA',
    device=DEVICE
)
client = OpenAI(
    api_key="ВАШ_API_КЛЮЧ",
    base_url="ВАШ_URL"
)

## Функции

In [59]:
def create_embeddings(model: SentenceTransformer, phrases: list[str]) -> dict[str, torch.Tensor]:
    encoded = [f"query: {phrase}" for phrase in phrases]
    embeddings = {}
    vectors = model.encode(encoded, convert_to_tensor=True, device=DEVICE, batch_size=16)

    for phrase, vec in zip(phrases, vectors):
        norm_vec = vec / torch.norm(vec)
        embeddings[phrase] = norm_vec

    print("Эмбеддинги посчитаны")
    return embeddings


def cluster_skills(embeddings: dict[str, torch.Tensor], eps=0.3, min_samples=2):
    phrases = list(embeddings.keys())
    vectors = torch.stack([embeddings[p] for p in phrases]).cpu().numpy()
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine').fit(vectors)

    clustered = defaultdict(list)
    for phrase, label in zip(phrases, clustering.labels_):
        clustered[label].append(phrase)

    print("Кластеризация выполнена")
    return clustered

In [60]:
def cluster_skills_llm(type, clusters: dict[int, list[str]], step):
    try:
        with open(f"{WORKING_DIRECTORY}/{step}/llm{step}{type}.json", "r", encoding="utf-8") as f:
            llm_ready = json.load(f)
            llm_ready[str(-1)] = None
    except:
        print("Исключение")
        llm_ready = {}
        llm_ready[str(-1)] = None
        with open(f"{WORKING_DIRECTORY}/{step}/llm{step}{type}.json", "w", encoding="utf-8") as f:
            json.dump(llm_ready, f, ensure_ascii=False, indent=2)

    cluster_groups = []
    current_group = {}
    for cid, phrases in clusters.items():
        str_cid = str(int(cid)) if isinstance(cid, (np.integer, np.int64)) else str(cid)
        if str_cid in llm_ready:
            continue
        current_group[str_cid] = phrases
        if len(current_group) >= 10:
            cluster_groups.append(current_group)
            current_group = {}

    if current_group:
        cluster_groups.append(current_group)

    for group in cluster_groups:
        try:
            prompt = (
                    "Ты — кластеризатор формулировок навыков.\n"
                    "Тебе будет предоставлен словарь, где ключи — идентификаторы кластеров, а значения — списки навыков.\n"
                    "Для каждого кластера (словаря) разбей список на подгруппы синонимов и дай каждой подгруппе короткое имя.\n"
                    "Выводи только JSON-объект, где ключи — исходные идентификаторы кластеров, а значения — массивы объектов {\"name\":...,\"items\":[...]}.\n"
                    "Пример:\n"
                    "Запрос: {\n"
                    "  \"1\": [\"разработка на python\", \"python разработчик\", \"pandas\"],\n"
                    "  \"2\": [\"java\", \"java программист\"]\n"
                    "}\n"
                    "Ответ: {\n"
                    "  \"1\": [{\"name\":\"python\",\"items\":[\"разработка на python\",\"python разработчик\"]}, {\"name\":\"pandas\",\"items\":[\"pandas\"]}],\n"
                    "  \"2\": [{\"name\":\"java\",\"items\":[\"java\", \"java программист\"]}]\n"
                    "}\n\n"
                    "Навыки:\n" + json.dumps(group, ensure_ascii=False)
            )
            response = client.chat.completions.create(model='gemini-2.5-flash',
                                                      messages=[{'role': 'user', 'content': prompt}])
            response_data = json.loads(response.choices[0].message.content)
            for cid, clusters in response_data.items():
                llm_ready[str(cid)] = clusters
                print(f"Готово: {cid}")

        except Exception as e:
            print(f"Ошибка в группе кластеров: {e}")
            for cid in group.keys():
                llm_ready[str(cid)] = None

        with open(f"{WORKING_DIRECTORY}/{step}/llm{step}{type}.json", "w", encoding="utf-8") as f:
            json.dump(llm_ready, f, ensure_ascii=False, indent=2)

In [61]:
def replace(type: str, step):
    try:
        os.makedirs(f"{WORKING_DIRECTORY}/{step + 1}", exist_ok=True)

        llm_path = f"{WORKING_DIRECTORY}/{step}/llm{step}{type}.json"
        if not os.path.exists(llm_path):
            raise FileNotFoundError(f"LLM файл не найден: {llm_path}")

        with open(llm_path, encoding="utf-8") as f:
            llm_data = json.load(f)

        replacement_map = {}
        for cluster in llm_data.values():
            if cluster is None:
                continue
            for group in cluster:
                canonical = group["name"].strip()
                for item in group["items"]:
                    replacement_map[item.strip()] = canonical

        def normalize(skills_str):
            if not skills_str or skills_str.strip() == "":
                return ""
            return ";".join(
                replacement_map.get(skill.strip(), skill.strip())
                for skill in skills_str.split(";")
                if skill.strip()
            )

        input_path = f"{WORKING_DIRECTORY}/{step + 1}/extracted_skills{step + 1}.txt"
        output_path = input_path

        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Входной файл не найден: {input_path}")

        temp_lines = []
        with open(input_path, encoding="utf-8") as f_in:
            for line in f_in:
                line = line.strip()
                if not line or "|" not in line:
                    continue

                parts = line.split("|")
                if len(parts) != 3:
                    continue

                job_id = parts[0].strip()
                hard_skills = parts[1].strip()
                soft_skills = parts[2].strip()

                if type == "hard":
                    hard_skills = normalize(hard_skills)
                else:
                    soft_skills = normalize(soft_skills)

                temp_lines.append(f"{job_id}|{hard_skills}|{soft_skills}\n")

        with open(output_path, "w", encoding="utf-8") as f_out:
            f_out.writelines(temp_lines)

        skills_path = f"{WORKING_DIRECTORY}/{step}/{type}{step}.txt"
        if os.path.exists(skills_path):
            with open(skills_path, encoding="utf-8") as f:
                original_skills = {skill.strip() for skill in f if skill.strip()}

            normalized_skills = {
                replacement_map.get(skill, skill)
                for skill in original_skills
            }

            output_skills_path = f"{WORKING_DIRECTORY}/{step + 1}/{type}{step + 1}.txt"
            with open(output_skills_path, "w", encoding="utf-8") as f:
                for skill in sorted(normalized_skills):
                    f.write(f"{skill}\n")

        print("Нормализация навыков успешно завершена")

    except Exception as e:
        print(f"Ошибка при нормализации навыков: {str(e)}")
        raise

In [62]:
def csv_to_txt(input_csv_path: str, output_txt_path: str):
    output_dir = os.path.dirname(output_txt_path)
    os.makedirs(output_dir, exist_ok=True)

    df = pd.read_csv(input_csv_path, encoding='utf-8-sig').fillna("")

    with open(output_txt_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            line = f"{row['_id']} | {row['hard_skills']} | {row['soft_skills']}\n"
            f.write(line)

    print(f"Файл успешно конвертирован в {output_txt_path}")


def txt_to_csv(input_txt_path: str, output_csv_path: str, professions_csv_path: str):
    professions_df = pd.read_csv(professions_csv_path, dtype={'_id': str}, encoding='utf-8-sig')
    professions = professions_df.set_index('_id')['best_profession'].to_dict()
    data = []
    with open(input_txt_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split('|')
            if len(parts) >= 3:
                job_id = parts[0].strip()
                data.append({
                    '_id': job_id,
                    'best_profession': professions.get(job_id, ''),
                    'hard_skills': parts[1].strip(),
                    'soft_skills': parts[2].strip()
                })

    pd.DataFrame(data).to_csv(output_csv_path, index=False, encoding='utf-8-sig')
    print(f"Файл успешно конвертирован в {output_csv_path}")

## Начало работы

In [12]:
step = 0
csv_to_txt(SKILLS_PATH, f"{WORKING_DIRECTORY}/0/extracted_skills0.txt")
shutil.copy(HARD_PATH,
            f"{WORKING_DIRECTORY}/0/hard0.txt")
shutil.copy(SOFT_PATH,
            f"{WORKING_DIRECTORY}/0/soft0.txt")
print("Всё успешно перенесено, готово к работе")

Файл успешно конвертирован в results/0/extracted_skills0.txt
Всё успешно перенесено, готово к работе


## Рабочий цикл

In [63]:
with open(f"{WORKING_DIRECTORY}/{step}/hard{step}.txt", "r", encoding="utf-8") as f:
    hard = [line.strip() for line in f if line.strip()]
with open(f"{WORKING_DIRECTORY}/{step}/soft{step}.txt", "r", encoding="utf-8") as f:
    soft = [line.strip() for line in f if line.strip()]
hard = list(set(hard))
soft = list(set(soft))

In [66]:
embs = create_embeddings(model, hard)
clusters = cluster_skills(embs, eps=0.4)
clusters

Эмбеддинги посчитаны
Кластеризация выполнена


defaultdict(list,
            {np.int64(-1): ['медицинское оборудование',
              'Формирование и предоставление отчетности руководителю, по запросу',
              'Опыт работы с графическими редакторами 3ds Max',
              'лечение заболеваний',
              'физиология',
              'анатомия животных',
              'Коммерческие предложения',
              'сертификаты',
              'Корректировка',
              'CATIA',
              'Разработка плана лечения',
              'работа с продуктовой воронкой',
              'Фотореалистичная визуализация',
              'Практический опыт работы ветеринарным хирургом',
              'Доставка',
              'Приготовление кондитерских компонентов',
              'перечисление денежных средств',
              'постпродажное обслуживание',
              'составление писем',
              'создание трендовых риелс',
              'работа по теплой базе',
              'Туры',
              'работа с потоком клиентов в 

In [67]:
with open(f"{WORKING_DIRECTORY}/{step}/hard{step}_pickle.pkl", 'wb') as f:
    pickle.dump(clusters, f)

In [68]:
with open(f"{WORKING_DIRECTORY}/{step}/hard{step}_pickle.pkl", 'rb') as f:
    clusters = pickle.load(f)

In [69]:
cluster_skills_llm("hard", clusters, step)

Исключение
Готово: 0
Готово: 1
Готово: 2
Готово: 3
Готово: 4
Готово: 5
Готово: 6
Готово: 7
Готово: 8
Готово: 9
Готово: 10
Готово: 11
Готово: 12
Готово: 13
Готово: 14
Готово: 15
Готово: 16


In [70]:
os.makedirs(f"{WORKING_DIRECTORY}/{step + 1}", exist_ok=True)
shutil.copy(f"{WORKING_DIRECTORY}/{step}/extracted_skills{step}.txt",
            f"{WORKING_DIRECTORY}/{step + 1}/extracted_skills{step + 1}.txt")
replace("hard", step)

Нормализация навыков успешно завершена


In [73]:
embs = create_embeddings(model, soft)
clusters = cluster_skills(embs, eps=0.45)
clusters

Эмбеддинги посчитаны
Кластеризация выполнена


defaultdict(list,
            {np.int64(-1): ['Презентабельный внешний вид',
              'физическая крепость',
              'трудолюбие',
              'Быть на связи 14/7',
              'система мотивации сотрудников',
              'Многозадачность и работа с информацией',
              'ответы на вопросы покупателей',
              'Эмпатия',
              'Приглашение лекторов',
              'лидерские качества',
              'стабильность',
              'Умение избегать конфликтных ситуаций',
              'Обучаемость и адаптивность',
              'работа в постоянно изменяющейся среде',
              'Доведение до продажи',
              'Планирование работы',
              'Выявление потребностей',
              'Грамотность речи',
              'Внутренняя самодисциплина',
              'Доверительные отношения',
              'Совершенствование знаний',
              'вовлеченность в процесс и сферу деятельности',
              'Кураторская поддержка',
              

In [74]:
with open(f"{WORKING_DIRECTORY}/{step}/soft{step}_pickle.pkl", 'wb') as f:
    pickle.dump(clusters, f)

In [75]:
with open(f"{WORKING_DIRECTORY}/{step}/soft{step}_pickle.pkl", 'rb') as f:
    clusters = pickle.load(f)

In [76]:
cluster_skills_llm("soft", clusters, step)

Исключение
Готово: 0
Готово: 1
Готово: 2
Готово: 3
Готово: 4
Готово: 5
Готово: 6
Готово: 7
Готово: 8
Готово: 9
Готово: 10
Готово: 11


In [77]:
replace("soft", step)

Нормализация навыков успешно завершена


In [78]:
step += 1

## Завершение работы

In [79]:
txt_to_csv(f"{WORKING_DIRECTORY}/{step - 1}/extracted_skills{step - 1}.txt",
           f"{WORKING_DIRECTORY}/result.csv", SKILLS_PATH)

Файл успешно конвертирован в results/result.csv
